<a href="https://colab.research.google.com/github/sreegithub234/ExData_Plotting1/blob/master/Classifer_GPUversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


#commands to setup GPU in Windows and CUDA-NVIDIA

'''
1 Download the Python(version) - Add path to the enviromnet variables
2 pip install virtualenv
3 virtualenv Name
4 Name/Scripts/activate.bat
5 Install the necessary pip packages:
      Ex: pip install Pillow == version
6 deactivate

#GPU drivers

1. Go to Nvidia driver website, select your GPU model, Download the driver file and install it.

'''

import os
import torch
import cv2  #opencv-python
import torchvision 
import torchvision.transforms as transforms #image-preprocessing

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

print(torch.__version__)

#Data-Preprocessing

transform = transforms.Compose(
	[transforms.ToTensor()])      #convert any input to this into a Torch format

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='/.data', train= True, download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root='./data', train = False, download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers =2)


testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers =2)


classes = ('plane', 'car', 'bird','cat', 'deer','dog', 'frog','horse','ship','truck')

# def imshow(img):

# 	npimg = img.numpy()
# 	img = cv2.imread(npimg)
# 	cv2.imshow("frame",img)
# 	cv2.waitKey(0)

class Classifier(nn.Module):

	def __init__(self):
		super().__init__()

		self.conv1 = nn.Conv2d(3, 6, 5)
		self.pool1 = nn.MaxPool2d(2,2)
		self.conv2 = nn.Conv2d(6,16,5)
		self.pool2 = nn.MaxPool2d(2,2)
		self.fc1 = nn.Linear(16*5*5,120)
		self.fc2 = nn.Linear(120,80)
		self.fc3 = nn.Linear(80,10)

	def forward(self, x):

		x = self.pool1(F.relu(self.conv1(x)))
		x = self.pool2(F.relu(self.conv2(x)))

		x = x.view(-1, 16*5*5)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)

net = Classifier()

print(net)

#define optimizer and losss


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum =0.9)


#Train the network:
for epoch in range(2):

	running_loss = 0.0

	for i, data in enumerate(trainloader,0):

		inputs, label = data

		optimizer.zero_grad()

		outputs = net(inputs)

		loss = criterion(outputs, label)

		loss.backward()
		optimizer.step()

		running_loss += loss.item()

		if i%2000 == 1999:
			print(epoch + 1, i+1, running_loss/2000+"\n")

			running_loss = 0.0

print("finished training")


Path = 'classifier.pth'
torch.save(net.state_dict(), Path)

print("\n It is saved")
